# はじめに
- TechFlow Navigatorは特定分野の社会・技術動向を提示してくれるAIエージェント

## 注意
- OPENAIのAPIを使う場合Emdebbingしているため費用が発生する。
    - ColaboratoryのSecret KeyとしてOPENAI_API_KEYを指定しなければ無料のSentenceTransformerが利用される


## 全体フロー
--- 1. 論文取得モジュール ---  
API経由でarXivから特定テーマの情報収集  

--- 2. Embedding生成モジュール ---
Embeddingを生成しベクトルDBへ投入  

--- 3. 系譜推定モジュール ---    

--- 4. LLMによる解説生成モジュール (Switchable) ---  

--- 5. ユーティリティ: 結果表示  ---  


In [4]:
!pip install feedparser
!pip install openai pandas tqdm
!pip install sentence-transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=b63962b394207c635fbe14ea3d75a96ae06985ae0ff00caef0b0458ce40486bc
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [9]:
from google.colab import userdata
import os
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    pass # OPENAI_API_KEY未設定時はMock版(定型文の解説を出力)として動作

import requests
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import feedparser
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

# Libs

In [10]:
# --- 1. 論文取得モジュール ---
def fetch_arxiv_papers(query="scaling law", max_results=30):
    """
    arXivのAPIを使用して論文情報を取得する。

    Args:
        query (str): 検索クエリ（例: "LLM scaling law"）。
        max_results (int): 取得する最大論文数。

    Returns:
        list: 取得した論文データ（ID、タイトル、サマリー、日付など）のリスト。
              取得失敗時は空のリストを返す。
    """
    base_url = "http://export.arxiv.org/api/query"

    # 検索パラメータ
    params = {
        "search_query": f"all:{query}",
        "start": 0,
        "max_results": max_results,
        # "sortBy": "submittedDate",
        # "sortOrder": "descending" # 新しいものが先に取得されることが多い
    }

    print(f"Fetching papers for query: '{query}'...")
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Error: API returned status {response.status_code}")
        return []

    feed = feedparser.parse(response.text)

    papers = []
    for entry in feed.entries:
        papers.append({
            "id": entry.get("id"),
            "title": entry.get("title").replace("\n", " "),
            "summary": entry.get("summary").replace("\n", " "),
            "authors": [a.name for a in entry.get("authors", [])],
            "published": entry.get("published"),
            "updated": entry.get("updated")
        })

    print(f"Fetched {len(papers)} papers.")
    return papers

# --- 2. Embedding生成モジュール ---
def generate_embeddings(papers):
    """
    論文のTitle/AbstractからEmbeddingを生成し、リストに追加する。

    Args:
        papers (list): 論文データ（'title', 'summary'を含む）のリスト。

    Returns:
        list: 'embedding'キーが付加された論文データのリスト。
    """
    print("Generating embeddings (this may take a moment)...")

    # タイトルとサマリーを結合して特徴量とする
    texts = [f"{p['title']} {p['summary']}" for p in papers]

    # モデル読み込み（MVPでは軽量な公開モデルを使用）
    model = SentenceTransformer("all-MiniLM-L6-v2")

    embeddings = model.encode(texts, batch_size=8, show_progress_bar=True)

    # Embeddingをリスト形式で各論文データに追加
    for p, emb in zip(papers, embeddings):
        p["embedding"] = emb.tolist()

    return papers

# --- 3. 系譜推定モジュール ---
def parse_arxiv_date(date_str: str) -> datetime:
    """
    arXivの日付文字列をdatetimeオブジェクトに変換する。

    Args:
        date_str (str): arXivの日付文字列（例: '2020-01-23T00:00:00Z'）。

    Returns:
        datetime: 変換されたdatetimeオブジェクト。
    """
    try:
        return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        return datetime.now() # パース失敗時は現在時刻を返すなどの対応

def estimate_lineage(papers, similarity_threshold=0.5):
    """
    時系列順に論文をソートし、過去の論文とのEmbedding類似度に基づいて親子関係（系譜）を推定する。

    Args:
        papers (list): 'embedding'と'published'キーを含む論文データのリスト。
        similarity_threshold (float): 親とみなす類似度の閾値（0.0～1.0）。

    Returns:
        list: 親子関係情報 ('parent_id', 'parent_title', 'relation_score') が付与され、
              時系列順にソートされた論文リスト。
    """
    print("Estimating lineage...")

    # 1. 時系列ソート
    for p in papers:
        p['_dt'] = parse_arxiv_date(p['published'])
    sorted_papers = sorted(papers, key=lambda x: x['_dt'])

    valid_papers = [p for p in sorted_papers if "embedding" in p]
    if not valid_papers:
        return []

    embeddings_matrix = np.array([p['embedding'] for p in valid_papers])

    # 2. 類似度行列の一括計算
    sim_matrix = cosine_similarity(embeddings_matrix)

    # 3. 親子関係の決定ロジック
    for i, current_paper in enumerate(valid_papers):
        current_paper['parent_id'] = None
        current_paper['parent_title'] = None
        current_paper['relation_score'] = 0.0

        if i == 0:
            continue

        # 過去の論文との類似度を取得
        past_sims = sim_matrix[i, :i]
        candidate_indices = np.where(past_sims >= similarity_threshold)[0]

        if len(candidate_indices) > 0:
            # 最も類似度が高いものを親とする
            best_idx = candidate_indices[np.argmax(past_sims[candidate_indices])]

            parent = valid_papers[best_idx]
            current_paper['parent_id'] = parent.get('id')
            current_paper['parent_title'] = parent.get('title')
            current_paper['relation_score'] = float(past_sims[best_idx])

    # 一時的な日付オブジェクトを削除
    for p in valid_papers:
        del p['_dt']

    return valid_papers


# --- 4. LLMによる解説生成モジュール (Switchable) ---
def _mock_explanation(score):
    """APIキーがない場合のダミー応答"""
    if score >= 0.7:
        return "パラダイムシフトとなる新構造を提案し、大幅な効率化を実現(Mock)"
    elif score >= 0.6:
        return "先行研究の課題であった計算コストを、新手法により削減(Mock)"
    else:
        return "類似の課題設定に対し、異なるデータセットを用いて検証(Mock)"

def _real_explanation_openai(parent_title, child_title, api_key):
    """OpenAI APIを使って解説を生成する"""
    client = OpenAI(api_key=api_key)

    prompt = f"""
    あなたは技術リサーチャーです。以下の2つの論文の関係性を分析してください。

    [親論文]: {parent_title}
    [子論文]: {child_title}

    指示: 親から子へ、技術的に何が進化したか、またはどう応用されたかを30文字程度の日本語で要約してください。
    """

    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=60,
            temperature=0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generation explanation: {str(e)}"

def generate_relation_explanation_dispatcher(parent_title, child_title, score):
    """
    環境変数を見てMockか本番かを自動で切り替えるディスパッチャー
    """
    api_key = os.getenv("OPENAI_API_KEY")

    if api_key:
        return _real_explanation_openai(parent_title, child_title, api_key)
    else:
        return _mock_explanation(score)

def process_llm_explanation(lineage_papers):
    """
    系譜データに対して解説文を生成・付与するメイン関数

    Args:
        lineage_papers (list): 系譜推定後の論文データのリスト。

    Returns:
        list: LLMによる解説 ('relation_description_by_llm') が付加された論文リスト。

    """
    # 実行モードの表示
    if os.getenv("OPENAI_API_KEY"):
        print("\nGenerating explanations using OpenAI API...")
    else:
        print("\nGenerating explanations using Mock (No API Key found)...")

    papers_with_parent = [p for p in lineage_papers if p.get('parent_id')]

    for i, p in enumerate(papers_with_parent):
        # 進捗表示もあると親切
        print(f"  - Explaining pair {i+1}/{len(papers_with_parent)}")

        # ディスパッチャー経由で呼び出すように変更
        explanation = generate_relation_explanation_dispatcher(
            p['parent_title'],
            p['title'],
            p['relation_score']
        )
        p['relation_description_by_llm'] = explanation

    return lineage_papers


# --- ユーティリティ: 結果表示  ---
def print_lineage_summary(papers: list):
    """
    推定された系譜とLLMによる解説をコンソールに出力する。

    Args:
        papers (list): 系譜情報とLLM解説が付与された論文データのリスト。
    """
    print("\n=== Lineage Summary (with LLM Explanations) ===")

    # 系譜のルートノードを識別し、ツリー構造に近い表示を試みる
    for p in papers:
        if p.get('parent_id'):
            # 子ノード
            date_str = p['published'][:10]
            title = p['title'][:50]

            # 親の情報
            score = p['relation_score']
            p_title = p['parent_title'][:40]
            llm_desc = p.get('relation_description_by_llm', '未生成')

            print("--------------------------------------------------")
            print(f"[{date_str}] {title}...")
            print(f"   ↑ ({score:.2f}) {llm_desc}")
            print(f"   └ From: {p_title}...")
        else:
            # ルートノード
            date_str = p['published'][:10]
            title = p['title'][:50]
            print(f"\n--- ROOT / Independent ---")
            print(f"[{date_str}] {title}...")


# Main

In [11]:
# --- メイン実行ブロック ---
if __name__ == "__main__":
    # 設定
    LLM_MODEL = "gpt-4o-mini"
    QUERY = "scaling law"
    MAX_RESULTS = 50
    SIMILARITY_THRESHOLD = 0.6
    OUTPUT_FILE = "arxiv_lineage_result.json"

    # 1. 取得
    papers = fetch_arxiv_papers(query=QUERY, max_results=MAX_RESULTS)

    if papers:
        # 2. Embedding生成
        papers = generate_embeddings(papers)

        # 3. 系譜推定
        lineage_papers = estimate_lineage(papers, similarity_threshold=SIMILARITY_THRESHOLD)

        # 4. コンソール表示
        print_lineage_summary(lineage_papers)

        # 5. LLMによる関係性の言語化 (New!)
        lineage_papers = process_llm_explanation(lineage_papers)

        # 6. JSON保存
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            saveable_papers = [{k: v for k, v in p.items() if k != 'embedding'} for p in lineage_papers]

            # データを保存
            json.dump(saveable_papers, f, indent=2, ensure_ascii=False)
        print(f"\nSaved results to {OUTPUT_FILE}")
    else:
        print("No papers found.")

Fetching papers for query: 'scaling law'...
Fetched 50 papers.
Generating embeddings (this may take a moment)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Estimating lineage...

=== Lineage Summary (with LLM Explanations) ===

--- ROOT / Independent ---
[1999-08-13] The Clustering of Faint Galaxies on Small Angular ...

--- ROOT / Independent ---
[2002-07-18] The Adiabatic Piston and the Second Law of Thermod...

--- ROOT / Independent ---
[2002-08-14] Why Do We Believe in the Second Law?...

--- ROOT / Independent ---
[2002-09-06] Observation of Mammalian Similarity Through Allome...

--- ROOT / Independent ---
[2002-11-17] A multidimensional Law of Sines...

--- ROOT / Independent ---
[2006-03-30] Scaling laws in the spatial structure of urban roa...

--- ROOT / Independent ---
[2007-02-18] Modelling Sex Ratio and Numbers for Translocation ...

--- ROOT / Independent ---
[2008-02-13] Spin coefficients for four-dimensional neutral met...

--- ROOT / Independent ---
[2009-08-05] A refined Tully-Fisher relationship and a new scal...

--- ROOT / Independent ---
[2010-01-21] A stability analysis of the power-law steady state...

--- ROOT / 